In [1]:
import numpy as np 
import pandas as pd 
import datetime
from datetime import datetime, date
from tqdm import tqdm_notebook as tqdm
import os


from multiprocessing import Pool
from functools import partial
import ALL_STOCK_preprocess_function as func

In [2]:
file_path = 'D:\\OneDrive - SinoPac\\Cmoney資料表\\基本面\\下市櫃公司基本資料\\'
file_list = os.listdir(file_path)
save_path = 'D:\\庫存健診開發\\data\\下市櫃公司基本資料\\'
df_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.read_off_stock, file_path=file_path, save_path=save_path), file_list), total=len(file_list)), 1):
                df_list.append(x)
                
off = pd.concat(df_list, axis=0)
off.to_csv('D:\\庫存健診開發\\data\\processed\\Offdate.csv', index=False)

In [3]:
file_path = 'D:\\OneDrive - SinoPac\\Cmoney資料表\\基本面\\上市櫃公司基本資料\\'
file_list = os.listdir(file_path)
save_path = 'D:\\庫存健診開發\\data\\上市櫃公司基本資料\\'
df_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.read_fundamental, file_path=file_path, save_path=save_path), file_list), total=len(file_list)), 1):
                df_list.append(x)
                
fundamental = pd.concat(df_list, axis=0)
fundamental.to_csv('D:\\庫存健診開發\\data\\processed\\Ondate.csv', index=False)

In [4]:
file_path = 'D:\\SinoPac\\彭偉綸-數位金融部-證券 - Cmoney資料表\\技術面\\日收盤還原表排行\\'
file_list = os.listdir(file_path)
save_path = 'D:\\庫存健診開發\\data\\價量資料\\'
df_list, index_list, industry_list = [], [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.data_preprocess, file_path=file_path, save_path=save_path), file_list), total=len(file_list)), 1):
            df_list.append(x[0])
            index_list.append(x[1])
            industry_list.append(x[2])
                
data, index, industry_index = pd.concat(df_list, axis=0), pd.concat(index_list, axis=0), pd.concat(industry_list, axis=0)
data['ts'], index['ts'], industry_index['ts'] = pd.to_datetime(data['ts']), pd.to_datetime(index['ts']), pd.to_datetime(industry_index['ts'])
index.to_csv('D:\\庫存健診開發\\data\\processed\\index.csv', index=False)
industry_index.to_csv('D:\\庫存健診開發\\data\\processed\\industry_index.csv', index=False)
data.to_csv('D:\\庫存健診開發\\data\\processed\\TWSE_ALL.csv', index=False)

In [5]:
df = func.filter_stocks(data, fundamental, off)
df

D:\庫存健診開發\code\Preprocess\ALL_STOCK_preprocess_function.py:155: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df.ts.dt.date > date(2007,7,1)][df.ts.dt.date < date(2019,9,24)]


,ts,StockNo,StockName,open,high,low,close,vol,total,capital,VWAP,On_Date
843,2007-07-02,1101,台泥,16.68,16.68,16.15,16.37,10222.0,381140.0,1202.8,37.286245,1962-02-09
844,2007-07-06,1101,台泥,16.15,16.87,16.04,16.83,25514.0,959258.0,1236.5,37.597319,1962-02-09
845,2007-07-03,1101,台泥,16.46,16.50,16.18,16.33,9831.0,367582.0,1199.6,37.390093,1962-02-09
846,2007-07-05,1101,台泥,16.46,16.59,16.15,16.15,15043.0,559331.0,1186.7,37.182145,1962-02-09
847,2007-07-09,1101,台泥,17.03,17.16,16.74,16.94,20613.0,801561.0,1244.5,38.886188,1962-02-09
...,...,...,...,...,...,...,...,...,...,...,...,...
3013648,2019-09-18,4571,鈞興-KY,90.70,97.00,89.40,94.30,1597.0,146349.0,44.9,91.639950,2019-09-17
3013649,2019-09-19,4571,鈞興-KY,92.80,93.90,91.40,92.00,806.0,74570.0,43.8,92.518610,2019-09-17
3013650,2019-09-17,4571,鈞興-KY,89.00,92.30,85.50,90.00,2753.0,243282.0,42.8,88.369778,2019-09-17
3013651,2019-09-20,4571,鈞興-KY,91.70,92.60,90.30,92.40,617.0,56557.0,44.0,91.664506,2019-09-17


In [10]:
file_path = 'D:\\SinoPac\\彭偉綸-數位金融部-證券 - Cmoney資料表\\基本面\\減資狀況\\'
file_list = os.listdir(file_path)
save_path = 'D:\\庫存健診開發\\data\\減資資料\\'
df_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.read_eliminate, file_path=file_path, save_path=save_path), file_list), total=len(file_list)), 1):
                df_list.append(x)
                
eliminate = pd.concat(df_list, axis=0)
eliminate = eliminate.sort_values(by='year')
eliminate.to_csv('D:\\庫存健診開發\\data\\processed\\eliminate.csv', index=False)

In [12]:
file_path = 'D:\\SinoPac\\彭偉綸-數位金融部-證券 - Cmoney資料表\\籌碼面\\日法人持股統計\\'
file_list = os.listdir(file_path)
save_path = 'D:\\庫存健診開發\\data\\三大法人\\'
df_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.read_inventory, file_path=file_path, save_path=save_path), file_list), total=len(file_list)), 1):
                df_list.append(x)
                
inventory = pd.concat(df_list, axis=0)
inventory = inventory.sort_values(by='ts')
inventory['ts'] = pd.to_datetime(inventory['ts'])
inventory.to_csv('D:\\庫存健診開發\\data\\processed\\inventory.csv', index=False)

In [19]:
inventory

,ts,StockNo,foreign_buy,investment_buy,dealer_buy
0,2004-02-10,0001,0.0,0.0,0
791,2004-02-10,5452,0.0,0.0,0
790,2004-02-10,5450,0.0,0.0,0
789,2004-02-10,5443,0.0,0.0,0
788,2004-02-10,5442R,0.0,0.0,0
...,...,...,...,...,...
1343,2019-10-29,5258,-102.0,0.0,-2
1344,2019-10-29,5259,0.0,0.0,0
1345,2019-10-29,5263,35.0,-70.0,-11
1332,2019-10-29,5213,-8.0,0.0,0


In [18]:
industry_index

,ts,reference,industry_open,industry_high,industry_low,industry_close,industry_vol
1159,2004-02-10,TWB11,63.78,66.04,63.52,65.84,262784.0
1160,2004-02-10,TWB12,327.39,338.68,327.39,335.39,203248.0
1161,2004-02-10,TWB13,126.46,128.39,126.19,127.91,373597.0
1162,2004-02-10,TWB14,273.56,277.27,272.06,277.27,556793.0
1163,2004-02-10,TWB15,87.23,88.25,86.90,87.63,241244.0
...,...,...,...,...,...,...,...
2107,2019-10-25,TWB37,111.80,112.07,111.00,111.27,355522.0
2108,2019-10-25,TWB38,113.71,114.00,113.40,113.91,19683.0
2109,2019-10-25,TWB39,117.46,118.02,116.94,116.97,6802.0
2110,2019-10-25,TWB40,81.31,81.42,80.56,80.90,73726.0
